In [2]:
import websocket
import json

In [3]:
currency = "ethusdt"
interval = "1m"
socket = f"wss://stream.binance.com:9443/ws/{currency}/@kline_{interval}"

In [4]:
def on_message(ws, message):
    print("Message reçu :", message)

def on_error(ws, error):
    print("Erreur :", error)

def on_close(ws, close_status_code, close_msg):
    print("### Connexion fermée ###")

def on_open(ws):
    print("Connexion ouverte")

In [5]:
ws = websocket.WebSocketApp(socket,
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

In [6]:
ws.run_forever()

Connexion ouverte
Erreur : 
### Connexion fermée ###


True

In [ ]:
import websocket
import json
import plotly.graph_objects as go
from datetime import datetime
from collections import deque
import plotly.io as pio

# Assurez-vous que Plotly utilise le navigateur pour le rendu
pio.renderers.default = 'browser'

# Configuration du WebSocket
currency = "ethusdt"
interval = "1m"
socket = f"wss://stream.binance.com:9443/ws/{currency}@kline_{interval}"

# Initialisation des données
times = deque(maxlen=100)
prices = deque(maxlen=100)

# Création du graphique
fig = go.Figure()
fig.add_trace(go.Scatter(x=[], y=[], name="Prix ETHUSDT"))
fig.show()

# Fonction pour mettre à jour le graphique
def update_graph():
    fig.data[0].x = list(times)
    fig.data[0].y = list(prices)
    if len(times) > 1:
        fig.update_layout(xaxis_range=[min(times), max(times)], yaxis_range=[min(prices), max(prices)])
    print("Graphique mis à jour avec les données :", list(times), list(prices))

# Fonctions de gestion des événements WebSocket
def on_message(ws, message):
    data = json.loads(message)
    candle = data['k']
    is_candle_closed = candle['x']
    if is_candle_closed:
        time = datetime.fromtimestamp(candle['t'] / 1000)
        price = float(candle['c'])

        times.append(time)
        prices.append(price)

        print(f"Données reçues - Temps: {time}, Prix: {price}")

        if len(times) > 1:
            update_graph()

def on_error(ws, error):
    print("Erreur :", error)

def on_close(ws, close_status_code, close_msg):
    print("### Connexion fermée ###")

def on_open(ws):
    print("Connexion ouverte")

# Connexion WebSocket
ws = websocket.WebSocketApp(socket,
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

ws.run_forever()


Connexion ouverte
Erreur : 
### Connexion fermée ###


True

In [ ]:
import websocket
import json
from datetime import datetime

# Configuration du WebSocket
currency = "ethusdt"
interval = "1s"
socket = f"wss://stream.binance.com:9443/ws/{currency}@kline_{interval}"

# Liste pour stocker toutes les valeurs
all_data = []

# Fonction pour sauvegarder les données dans un fichier JSON
def save_to_json():
    with open('ethusdt_prices.json', 'w') as json_file:
        json.dump(all_data, json_file, indent=4)

# Fonctions de gestion des événements WebSocket
def on_message(ws, message):
    data = json.loads(message)
    candle = data['k']
    is_candle_closed = candle['x']
    if is_candle_closed:
        time = datetime.fromtimestamp(candle['t'] / 1000)
        price = float(candle['c'])

        # Création d'un dictionnaire avec les données
        data_dict = {
            "Temps": time.isoformat(),
            "Prix": price
        }

        # Ajout des données à la liste
        all_data.append(data_dict)

        # Sauvegarde des données dans un fichier JSON
        save_to_json()
        print(f"Données sauvegardées : {data_dict}")

def on_error(ws, error):
    print("Erreur :", error)

def on_close(ws, close_status_code, close_msg):
    print("### Connexion fermée ###")

def on_open(ws):
    print("Connexion ouverte")

# Connexion WebSocket
ws = websocket.WebSocketApp(socket,
                            on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close)

ws.run_forever()


Connexion ouverte
Données reçues - Temps: 2025-07-22 16:02:00, Prix: 3635.73
Données sauvegardées : {'Temps': '2025-07-22T16:02:59', 'Prix': 3635.73}
Données sauvegardées : {'Temps': '2025-07-22T16:03:00', 'Prix': 3636.22}
Données sauvegardées : {'Temps': '2025-07-22T16:03:01', 'Prix': 3636.36}
Données sauvegardées : {'Temps': '2025-07-22T16:03:02', 'Prix': 3635.97}
Données sauvegardées : {'Temps': '2025-07-22T16:03:03', 'Prix': 3636.58}
Données sauvegardées : {'Temps': '2025-07-22T16:03:04', 'Prix': 3635.5}
Données sauvegardées : {'Temps': '2025-07-22T16:03:05', 'Prix': 3634.55}
Données sauvegardées : {'Temps': '2025-07-22T16:03:06', 'Prix': 3634.01}
Données sauvegardées : {'Temps': '2025-07-22T16:03:07', 'Prix': 3635.29}
Données sauvegardées : {'Temps': '2025-07-22T16:03:08', 'Prix': 3636.18}
Données sauvegardées : {'Temps': '2025-07-22T16:03:09', 'Prix': 3636.54}
Données sauvegardées : {'Temps': '2025-07-22T16:03:10', 'Prix': 3636.65}
Données sauvegardées : {'Temps': '2025-07-22T16:

True

In [1]:
import json
import threading
import time
from collections import deque
from datetime import datetime, timezone
import websocket
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# ------------------------------
# Configuration
# ------------------------------
SYMBOL = "ethusdt"
INTERVAL = "1s"
WS_URL = f"wss://stream.binance.com:9443/ws/{SYMBOL}@kline_{INTERVAL}"
MAX_POINTS = 300  # nombre max de points à afficher

times = deque(maxlen=MAX_POINTS)
prices = deque(maxlen=MAX_POINTS)
lock = threading.Lock()

# ------------------------------
# WebSocket handlers
# ------------------------------
def on_message(ws, message):
    data = json.loads(message)
    candle = data.get("k", {})
    if candle.get("x"):  # bougie fermée
        ts_ms = candle["t"]
        price = float(candle["c"])
        dt = datetime.fromtimestamp(ts_ms / 1000, tz=timezone.utc)
        with lock:
            times.append(dt)
            prices.append(price)

def on_error(ws, error):
    print("WebSocket error:", error)

def on_close(ws, close_status_code, close_msg):
    print("WebSocket closed")

def on_open(ws):
    print("WebSocket connected")

def ws_thread():
    while True:
        ws = websocket.WebSocketApp(
            WS_URL,
            on_open=on_open,
            on_message=on_message,
            on_error=on_error,
            on_close=on_close,
        )
        try:
            ws.run_forever(ping_interval=20, ping_timeout=10)
        except Exception as e:
            print("Exception in run_forever:", e)
        print("Reconnecting in 5 seconds...")
        time.sleep(5)

# Lance le thread websocket en daemon (arrière-plan)
threading.Thread(target=ws_thread, daemon=True).start()

# ------------------------------
# Dash app
# ------------------------------
app = dash.Dash(__name__)
app.title = "ETH/USDT Live Price"

app.layout = html.Div([
    html.H2(f"Prix en direct {SYMBOL.upper()} ({INTERVAL} kline)"),
    dcc.Graph(id="live-graph"),
    dcc.Interval(id="update-interval", interval=1000, n_intervals=0),
    html.Div(id="last-price", style={"fontSize": "18px", "marginTop": "10px"}),
], style={"maxWidth": "900px", "margin": "auto", "fontFamily": "Arial, sans-serif"})

@app.callback(
    [Output("live-graph", "figure"),
     Output("last-price", "children")],
    [Input("update-interval", "n_intervals")]
)
def update_graph_live(_):
    with lock:
        x = list(times)
        y = list(prices)

    if not x:
        fig = go.Figure()
        last_price_text = "En attente des données..."
    else:
        fig = go.Figure(data=[
            go.Scatter(x=x, y=y, mode="lines+markers", name="Prix clôture")
        ])
        fig.update_layout(
            title="Évolution du prix ETH/USDT",
            xaxis_title="Temps (UTC)",
            yaxis_title="Prix (USDT)",
            template="plotly_white",
            xaxis=dict(tickformat="%H:%M:%S", rangeslider=dict(visible=False)),
            margin=dict(l=40, r=20, t=40, b=40),
        )
        last_price_text = f"Dernier prix : {y[-1]:.2f} USDT à {x[-1].strftime('%H:%M:%S UTC')}"

    return fig, last_price_text

# ------------------------------
# Point d'entrée
# ------------------------------
if __name__ == "__main__":
    # Lance le serveur Dash en local sur 127.0.0.1:8050
    app.run(debug=True)


WebSocket connected


In [2]:
import json
from datetime import datetime
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

HISTORIC_JSON_FILE = "anciennes_valeurs.json"

def load_data_from_json(filename):
    with open(filename, "r") as f:
        return json.load(f)

def process_klines_data(klines):
    times = []
    prices = []
    for k in klines:
        dt = datetime.fromtimestamp(k[0] / 1000)
        price = float(k[4])
        times.append(dt)
        prices.append(price)
    return times, prices

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Historique des prix ETHUSDT sur la dernière journée"),
    html.Button("Recharger les données", id="reload-button", n_clicks=0),
    dcc.Graph(id="historical-graph"),
])

@app.callback(
    Output("historical-graph", "figure"),
    Input("reload-button", "n_clicks")
)
def update_graph(n_clicks):
    # Chargement et traitement des données à chaque clic
    klines = load_data_from_json(HISTORIC_JSON_FILE)
    times, prices = process_klines_data(klines)
    fig = go.Figure(data=[go.Scatter(x=times, y=prices, mode="lines+markers", name="Prix")])
    fig.update_layout(xaxis_title="Temps", yaxis_title="Prix (USDT)", template="plotly_white")
    return fig

if __name__ == "__main__":
    app.run(debug=True)


In [3]:
import json
from datetime import datetime
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State

HISTORIC_JSON_FILE = "anciennes_valeurs.json"

def load_data_from_json(filename):
    with open(filename, "r") as f:
        return json.load(f)

def process_klines_data(klines):
    times = []
    prices = []
    for k in klines:
        dt = datetime.fromtimestamp(k[0] / 1000)
        price = float(k[4])
        times.append(dt)
        prices.append(price)
    return times, prices

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Historique des prix ETHUSDT sur la dernière journée"),
    html.Button("Recharger les données", id="reload-button", n_clicks=0),
    dcc.Graph(id="historical-graph"),
    html.Div("Valeurs mises à jour !", id="popup-message", style={
        "display": "none",
        "position": "fixed",
        "top": "20px",
        "right": "20px",
        "background-color": "lightgreen",
        "padding": "10px",
        "border-radius": "5px",
        "box-shadow": "0 0 5px #333",
        "zIndex": 1000,
    }),
    dcc.Interval(id="popup-interval", interval=2000, n_intervals=0, max_intervals=1, disabled=True),
])

@app.callback(
    [Output("historical-graph", "figure"),
     Output("popup-message", "style"),
     Output("popup-interval", "disabled"),
     Output("popup-interval", "n_intervals")],
    [Input("reload-button", "n_clicks"),
     Input("popup-interval", "n_intervals")],
    [State("popup-message", "style")],
)
def update_graph_and_popup(n_clicks, n_intervals, current_style):
    ctx = dash.callback_context

    if not ctx.triggered:
        # Au chargement de la page
        return dash.no_update, {"display": "none"}, True, 0

    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0]

    if triggered_id == "reload-button" and n_clicks > 0:
        # Bouton cliqué : charge données, affiche popup et active intervalle
        klines = load_data_from_json(HISTORIC_JSON_FILE)
        times, prices = process_klines_data(klines)

        fig = go.Figure(data=[go.Scatter(x=times, y=prices, mode="lines+markers", name="Prix")])
        fig.update_layout(xaxis_title="Temps", yaxis_title="Prix (USDT)", template="plotly_white")

        style_show = {
            "display": "block",
            "position": "fixed",
            "top": "20px",
            "right": "20px",
            "background-color": "lightgreen",
            "padding": "10px",
            "border-radius": "5px",
            "box-shadow": "0 0 5px #333",
            "zIndex": 1000,
        }
        return fig, style_show, False, 0

    elif triggered_id == "popup-interval" and n_intervals == 1:
        # Intervalle fini : cache popup, désactive intervalle
        return dash.no_update, {"display": "none"}, True, 0

    else:
        # Pas de changement
        return dash.no_update, current_style, dash.no_update, dash.no_update

if __name__ == "__main__":
    app.run(debug=True)
